# 02 - Prompt Engineering Experiments

Evaluate 7 prompt strategies on the Qwen2-VL base model for CORD receipt extraction.

**Strategies tested:**
1. Zero-shot basic - minimal instruction
2. Zero-shot detailed - field categories described
3. Zero-shot structured - explicit JSON schema
4. Few-shot (2 examples) - two representative examples
5. Few-shot (5 examples) - five diverse examples
6. Chain-of-thought step-by-step - guided reasoning
7. Chain-of-thought self-verify - extract then verify

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import torch
from pathlib import Path
from rich.console import Console
from rich.table import Table

from src.config import load_base_config
from src.data.cord_loader import load_cord_dataset, get_cord_schema
from src.data.format_converter import sample_to_inference_chatml
from src.prompts.templates import ReceiptExtractionTemplate, get_template, list_templates
from src.prompts.strategies import (
    ZeroShotBasic,
    ZeroShotDetailed,
    ZeroShotStructured,
    FewShotStrategy2,
    FewShotStrategy5,
    ChainOfThoughtStepByStep,
    ChainOfThoughtSelfVerify,
    get_all_strategies,
    list_strategies,
)
from src.prompts.experiment_runner import (
    PromptExperimentRunner,
    extract_json_from_response,
)

console = Console()
config = load_base_config()
print(f'Project: {config.project.name}')
print(f'Model:   {config.model.name}')
print(f'Available strategies: {list_strategies()}')
print(f'Available templates:  {list_templates()}')

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor

model_name = config.model.name
print(f'Loading model: {model_name}')

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    trust_remote_code=True,
)
model.eval()

device = next(model.parameters()).device
print(f'Model loaded on device: {device}')
print(f'Model dtype: {next(model.parameters()).dtype}')

In [ ]:
# Load test samples and few-shot examples
test_samples = load_cord_dataset(split='test', max_samples=5)
example_samples = load_cord_dataset(split='train', max_samples=10)

print(f'Test samples:    {len(test_samples)}')
print(f'Example samples: {len(example_samples)}')

# Define all 7 strategies
strategies = [
    ZeroShotBasic(),
    ZeroShotDetailed(),
    ZeroShotStructured(),
    FewShotStrategy2(example_samples=example_samples),
    FewShotStrategy5(example_samples=example_samples),
    ChainOfThoughtStepByStep(),
    ChainOfThoughtSelfVerify(),
]

print(f'\nDefined {len(strategies)} strategies:')
for s in strategies:
    print(f'  - {s.get_name()}')

# Preview the prompt for the first strategy on the first sample
print(f'\n--- Sample prompt for "{strategies[0].get_name()}" ---')
print(strategies[0].build_prompt(test_samples[0]))
print('\n--- Sample prompt for "zero_shot_structured" ---')
print(strategies[2].build_prompt(test_samples[0]))

In [ ]:
# Run each strategy on 5 test samples
runner = PromptExperimentRunner(
    model=model,
    processor=processor,
    strategies=strategies,
    max_new_tokens=1024,
)

results = runner.run_experiment(
    test_samples=test_samples,
    num_samples=5,
)

# Show raw predictions for the first sample across all strategies
print('=== Predictions for first test sample ===')
print(f'Ground truth: {json.dumps(test_samples[0]["ground_truth"], indent=2, ensure_ascii=False)[:500]}')
print()

for strat_result in results['strategy_results']:
    name = strat_result['strategy']
    first = strat_result['per_sample'][0]
    print(f'--- {name} ---')
    print(f'JSON Valid: {first["metrics"]["json_valid"]}')
    print(f'F1: {first["metrics"]["f1"]:.4f}')
    output_preview = first['raw_output'][:300]
    print(f'Output (preview): {output_preview}')
    print()

In [ ]:
# Build comparison table
table = Table(
    title='Prompt Strategy Comparison (5 samples)',
    show_header=True,
    header_style='bold magenta',
)
table.add_column('Rank', style='dim', width=4, justify='right')
table.add_column('Strategy', style='cyan', min_width=22)
table.add_column('JSON Valid', justify='right')
table.add_column('Avg F1', justify='right')
table.add_column('Micro F1', justify='right')
table.add_column('Precision', justify='right')
table.add_column('Recall', justify='right')
table.add_column('Time (s)', justify='right')

for i, row in enumerate(results['comparison']):
    style = 'bold green' if i == 0 else None
    table.add_row(
        str(i + 1),
        row['strategy'],
        f"{row['json_valid_rate']:.0%}",
        f"{row['avg_f1']:.4f}",
        f"{row['micro_f1']:.4f}",
        f"{row['avg_precision']:.4f}",
        f"{row['avg_recall']:.4f}",
        f"{row['elapsed_seconds']:.1f}",
        style=style,
    )

console.print(table)

# Save results
output_path = Path('../outputs/prompt_experiments/notebook_results.json')
PromptExperimentRunner.save_results(results, output_path)
print(f'\nResults saved to: {output_path}')

## Analysis and Findings

### Key Observations

| Dimension | Finding |
|-----------|----------|
| **JSON Validity** | _[Fill in: Which strategies most reliably produce valid JSON?]_ |
| **Field Accuracy** | _[Fill in: Which strategy achieves the highest field-level F1?]_ |
| **Schema Guidance** | _[Fill in: How much does providing the JSON schema help?]_ |
| **Few-shot Scaling** | _[Fill in: Does 5-shot outperform 2-shot? By how much?]_ |
| **Chain-of-Thought** | _[Fill in: Does CoT reasoning improve extraction quality?]_ |
| **Self-Verification** | _[Fill in: Does the verify step catch and correct errors?]_ |
| **Latency** | _[Fill in: What is the latency cost of more complex prompts?]_ |

### Hypotheses for Next Steps

1. **Best base prompt for fine-tuning**: The strategy with the highest F1 should be used as the instruction during QLoRA fine-tuning.
2. **Schema guidance is likely critical**: Structured prompts should significantly outperform basic ones, confirming the need for schema in the training prompt.
3. **CoT may not help base model**: Without fine-tuning, the base model may not reliably follow multi-step reasoning, suggesting CoT benefits may only appear post-training.
4. **Few-shot examples help format compliance**: Even if field accuracy is similar, few-shot examples should improve JSON validity rates.

### Recommended Strategy for Phase 4 (Fine-tuning)

_[Fill in: Which strategy to use as the training instruction and why]_